In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop2.7'

In [ ]:
import sys
import os
import numpy as np
# !pip install --upgrade pyspark==3.1.1

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import year, last, count, isnan, when, col, isnull
from pyspark.sql.functions import to_date, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.window import Window

from sklearn.model_selection import train_test_split

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

from pyspark.sql import Window

In [ ]:
df1 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-1.csv")

df2 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-2.csv")

df3 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-3.csv")

df4 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-4.csv")
df5 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-5.csv")
df6 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-6.csv")

df7 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-7.csv")

df8 = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/train-8.csv")

df_val  = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/validation_hidden.csv")

df_test  = spark.read.option("header",True) \
     .csv("/content/drive/MyDrive/big data imbd/big-data-course-2022-projects-master/imdb/test_hidden.csv")

# Nieuwe sectie

In [ ]:
from functools import reduce

# union all dataframes into a single dataframe
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]
df_all = reduce(lambda df1, df2: df1.union(df2), dfs)

df_all.show()
count = df_all.count()
print("Number of entries in df_count:", count)


+---+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|_c0|   tconst|        primaryTitle|       originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|
+---+---------+--------------------+--------------------+---------+-------+--------------+--------+-----+
|  4|tt0010600|            The Doll|           Die Puppe|     1919|     \N|            66|  1898.0| True|
|  7|tt0011841|       Way Down East|       Way Down East|     1920|     \N|           145|  5376.0| True|
|  9|tt0012494|             Déstiny|        Der müde Tod|     1921|     \N|            97|  5842.0| True|
| 25|tt0015163|       The Navigator|       The Navigator|     1924|     \N|            59|  9652.0| True|
| 38|tt0016220|The Phantom of th...|The Phantom of th...|     1925|     \N|            93| 17887.0| True|
| 42|tt0016630|     Báttling Bútlér|     Battling Butler|     1926|     \N|            77|  3285.0| True|
| 81|tt0021015|Juno and the Paycock|          

In [ ]:
from pyspark.sql.functions import col, count, isnan, when

# recognize "\N" as Null
df_all = df_all.select([when(col(c)=="\\N",None).otherwise(col(c)).alias(c) for c in df_all.columns])
df_val = df_val.select([when(col(c)=="\\N",None).otherwise(col(c)).alias(c) for c in df_val.columns])
df_test = df_test.select([when(col(c)=="\\N",None).otherwise(col(c)).alias(c) for c in df_test.columns])

# count the number of missing values in each column
missing_values_all = df_all.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_all.columns])
missing_values_val = df_val.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_val.columns])
missing_values_test = df_test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_test.columns])


# display the results
missing_values_all.show()
missing_values_val.show()
missing_values_test.show()


+---+------+------------+-------------+---------+-------+--------------+--------+-----+
|_c0|tconst|primaryTitle|originalTitle|startYear|endYear|runtimeMinutes|numVotes|label|
+---+------+------------+-------------+---------+-------+--------------+--------+-----+
|  0|     0|           0|         3988|      786|   7173|            13|     790|    0|
+---+------+------------+-------------+---------+-------+--------------+--------+-----+

+---+------+------------+-------------+---------+-------+--------------+--------+
|_c0|tconst|primaryTitle|originalTitle|startYear|endYear|runtimeMinutes|numVotes|
+---+------+------------+-------------+---------+-------+--------------+--------+
|  0|     0|           0|          468|       94|    861|             2|      91|
+---+------+------------+-------------+---------+-------+--------------+--------+

+---+------+------------+-------------+---------+-------+--------------+--------+
|_c0|tconst|primaryTitle|originalTitle|startYear|endYear|runtimeMi

In [ ]:
# !pip install --upgrade pyspark==3.1.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
import pandas as pd
from pyspark.sql.functions import *

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, StringType,IntegerType

##Function that removes accent
@F.pandas_udf('string')
def strip_accents(s: pd.Series) -> pd.Series:
    return s.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [ ]:
## Fixing the titles and years
# Stripping accents
df_all = df_all.withColumn('primaryTitle', strip_accents('primaryTitle')).withColumn('originalTitle', strip_accents('originalTitle'))
df_val = df_val.withColumn('primaryTitle', strip_accents('primaryTitle')).withColumn('originalTitle', strip_accents('originalTitle'))
df_test = df_test.withColumn('primaryTitle', strip_accents('primaryTitle')).withColumn('originalTitle', strip_accents('originalTitle'))

# Copy the primaryTitle to the missing values of originalTitles, also copying the years
df_all = df_all.withColumn("originalTitle", coalesce(df_all["originalTitle"],df_all["primaryTitle"])).withColumn("endYear", coalesce(df_all["startYear"], df_all["endYear"])).drop("startYear")
df_val = df_val.withColumn("originalTitle", coalesce(df_val["originalTitle"],df_val["primaryTitle"])).withColumn("endYear", coalesce(df_val["startYear"], df_val["endYear"])).drop("startYear")
df_test = df_test.withColumn("originalTitle", coalesce(df_test["originalTitle"],df_test["primaryTitle"])).withColumn("endYear", coalesce(df_test["startYear"], df_test["endYear"])).drop("startYear")

df_all.show()
df_val.show()
df_test.show()

+---+---------+--------------------+--------------------+-------+--------------+--------+-----+
|_c0|   tconst|        primaryTitle|       originalTitle|endYear|runtimeMinutes|numVotes|label|
+---+---------+--------------------+--------------------+-------+--------------+--------+-----+
|  4|tt0010600|            The Doll|           Die Puppe|   1919|            66|  1898.0| True|
|  7|tt0011841|       Way Down East|       Way Down East|   1920|           145|  5376.0| True|
|  9|tt0012494|             Destiny|        Der mude Tod|   1921|            97|  5842.0| True|
| 25|tt0015163|       The Navigator|       The Navigator|   1924|            59|  9652.0| True|
| 38|tt0016220|The Phantom of th...|The Phantom of th...|   1925|            93| 17887.0| True|
| 42|tt0016630|     Battling Butler|     Battling Butler|   1926|            77|  3285.0| True|
| 81|tt0021015|Juno and the Paycock|Juno and the Paycock|   1929|            85|  2275.0|False|
|118|tt0023973|The Eagle and the...|The 

In [ ]:
from pyspark.sql.types import IntegerType, BooleanType
df_all = df_all.withColumn('runtimeMinutes',df_all['runtimeMinutes'].cast(IntegerType())).withColumn('numVotes',df_all['numVotes'].cast(IntegerType()))
for col in df_all.dtypes:
    print(col[0]+" , "+col[1])

df_val = df_val.withColumn('runtimeMinutes',df_val['runtimeMinutes'].cast(IntegerType())).withColumn('numVotes',df_val['numVotes'].cast(IntegerType()))
for col in df_val.dtypes:
    print(col[0]+" , "+col[1])

df_test = df_test.withColumn('runtimeMinutes',df_test['runtimeMinutes'].cast(IntegerType())).withColumn('numVotes',df_test['numVotes'].cast(IntegerType()))
for col in df_test.dtypes:
    print(col[0]+" , "+col[1])

_c0 , string
tconst , string
primaryTitle , string
originalTitle , string
endYear , string
runtimeMinutes , int
numVotes , int
label , string
_c0 , string
tconst , string
primaryTitle , string
originalTitle , string
endYear , string
runtimeMinutes , int
numVotes , int
_c0 , string
tconst , string
primaryTitle , string
originalTitle , string
endYear , string
runtimeMinutes , int
numVotes , int


In [ ]:
# Fixing the null in numVotes and runtimeMinutes
# Replace missing values with median in the runtime and numVotes column

all_runtime_med = df_all.approxQuantile('runtimeMinutes',[0.5],0.1)[0]
all_numvotes_med = df_all.approxQuantile('numVotes',[0.5],0.1)[0]
df_all = df_all.na.fill(all_runtime_med, subset=['runtimeMinutes']).na.fill(all_numvotes_med, subset=['numVotes'])

val_runtime_med = df_val.approxQuantile('runtimeMinutes',[0.5],0.1)[0]
val_numvotes_med = df_val.approxQuantile('numVotes',[0.5],0.1)[0]
df_val = df_val.na.fill(val_runtime_med, subset=['runtimeMinutes']).na.fill(val_numvotes_med, subset=['numVotes'])

test_runtime_med = df_test.approxQuantile('runtimeMinutes',[0.5],0.1)[0]
test_numvotes_med = df_test.approxQuantile('numVotes',[0.5],0.1)[0]
df_test = df_test.na.fill(test_runtime_med, subset=['runtimeMinutes']).na.fill(test_numvotes_med, subset=['numVotes'])

df_all.show()
df_val.show()
df_test.show()

+---+---------+--------------------+--------------------+-------+--------------+--------+-----+
|_c0|   tconst|        primaryTitle|       originalTitle|endYear|runtimeMinutes|numVotes|label|
+---+---------+--------------------+--------------------+-------+--------------+--------+-----+
|  4|tt0010600|            The Doll|           Die Puppe|   1919|            66|    1898| True|
|  7|tt0011841|       Way Down East|       Way Down East|   1920|           145|    5376| True|
|  9|tt0012494|             Destiny|        Der mude Tod|   1921|            97|    5842| True|
| 25|tt0015163|       The Navigator|       The Navigator|   1924|            59|    9652| True|
| 38|tt0016220|The Phantom of th...|The Phantom of th...|   1925|            93|   17887| True|
| 42|tt0016630|     Battling Butler|     Battling Butler|   1926|            77|    3285| True|
| 81|tt0021015|Juno and the Paycock|Juno and the Paycock|   1929|            85|    2275|False|
|118|tt0023973|The Eagle and the...|The 

In [ ]:
from pyspark.sql.functions import col, count, isnan, when

# count the number of missing values in each column
missing_values_all = df_all.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_all.columns])
missing_values_val = df_val.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_val.columns])
missing_values_test = df_test.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_test.columns])


# display the results
missing_values_all.show()
missing_values_val.show()
missing_values_test.show()



+---+------+------------+-------------+-------+--------------+--------+-----+
|_c0|tconst|primaryTitle|originalTitle|endYear|runtimeMinutes|numVotes|label|
+---+------+------------+-------------+-------+--------------+--------+-----+
|  0|     0|           0|            0|      0|             0|       0|    0|
+---+------+------------+-------------+-------+--------------+--------+-----+

+---+------+------------+-------------+-------+--------------+--------+
|_c0|tconst|primaryTitle|originalTitle|endYear|runtimeMinutes|numVotes|
+---+------+------------+-------------+-------+--------------+--------+
|  0|     0|           0|            0|      0|             0|       0|
+---+------+------------+-------------+-------+--------------+--------+

+---+------+------------+-------------+-------+--------------+--------+
|_c0|tconst|primaryTitle|originalTitle|endYear|runtimeMinutes|numVotes|
+---+------+------------+-------------+-------+--------------+--------+
|  0|     0|           0|       

In [ ]:
# from pyspark.ml.linalg import Vectors

# from pyspark.ml.linalg import Vectors

# # Convert features column to vector data type
# df_all = df_all.withColumn('features_vec', Vectors.dense(df_all.features.values))
# df_val = df_val.withColumn('features_vec', Vectors.dense(df_val.features.values))
# df_test = df_test.withColumn('features_vec', Vectors.dense(df_test.features.values))


# # Cast columns to double data type
# df_all = df_all.select([df_all[col].cast('double').alias(col) for col in df_all.columns])
# df_val = df_val.select([df_val[col].cast('double').alias(col) for col in df_val.columns])
# df_test = df_test.select([df_test[col].cast('double').alias(col) for col in df_test.columns])


In [ ]:
df_all.where(df_all.label.isNull() | isnan(df_all.label)).count()


0

In [ ]:
df_all.show()

# Model

In [ ]:
from pyspark.ml.feature import VectorAssembler

numericCols = ['numVotes','runtimeMinutes']
# allCols = ['primaryTitle','originalTitle','endYear','runtimeMinutes','numVotes']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df_all = assembler.transform(df_all)
df_all.show()

+---+---------+--------------------+--------------------+-------+--------------+--------+-----+---------------+
|_c0|   tconst|        primaryTitle|       originalTitle|endYear|runtimeMinutes|numVotes|label|       features|
+---+---------+--------------------+--------------------+-------+--------------+--------+-----+---------------+
|  4|tt0010600|            The Doll|           Die Puppe|   1919|            66|    1898| True|  [1898.0,66.0]|
|  7|tt0011841|       Way Down East|       Way Down East|   1920|           145|    5376| True| [5376.0,145.0]|
|  9|tt0012494|             Destiny|        Der mude Tod|   1921|            97|    5842| True|  [5842.0,97.0]|
| 25|tt0015163|       The Navigator|       The Navigator|   1924|            59|    9652| True|  [9652.0,59.0]|
| 38|tt0016220|The Phantom of th...|The Phantom of th...|   1925|            93|   17887| True| [17887.0,93.0]|
| 42|tt0016630|     Battling Butler|     Battling Butler|   1926|            77|    3285| True|  [3285.0

In [ ]:
from pyspark.ml.feature import StringIndexer

label_stringIdx = StringIndexer(inputCol = 'label', outputCol = 'labelIndex')
df_all = label_stringIdx.fit(df_all).transform(df_all)
df_all.show()

+---+---------+--------------------+--------------------+-------+--------------+--------+-----+---------------+----------+
|_c0|   tconst|        primaryTitle|       originalTitle|endYear|runtimeMinutes|numVotes|label|       features|labelIndex|
+---+---------+--------------------+--------------------+-------+--------------+--------+-----+---------------+----------+
|  4|tt0010600|            The Doll|           Die Puppe|   1919|            66|    1898| True|  [1898.0,66.0]|       0.0|
|  7|tt0011841|       Way Down East|       Way Down East|   1920|           145|    5376| True| [5376.0,145.0]|       0.0|
|  9|tt0012494|             Destiny|        Der mude Tod|   1921|            97|    5842| True|  [5842.0,97.0]|       0.0|
| 25|tt0015163|       The Navigator|       The Navigator|   1924|            59|    9652| True|  [9652.0,59.0]|       0.0|
| 38|tt0016220|The Phantom of th...|The Phantom of th...|   1925|            93|   17887| True| [17887.0,93.0]|       0.0|
| 42|tt0016630| 

In [ ]:
train,test = df_all.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 5611
Test Dataset Count: 2348


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'labelIndex')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('numVotes','runtimeMinutes').show(25)

+--------+--------------+
|numVotes|runtimeMinutes|
+--------+--------------+
|    2705|            65|
|    1444|            84|
|    2728|           111|
|    2841|            90|
|    9110|           107|
|    2841|            73|
|    1498|            98|
|    3125|           125|
|    2763|            94|
|    6943|           108|
|    4616|            88|
|    1577|            81|
|    1741|            90|
|    4390|           202|
|  133304|            93|
|    2841|            80|
|    5265|            90|
|    5776|            95|
|    2841|            86|
|   11419|            97|
|  636802|           147|
|   53698|            95|
|    8828|           104|
|    1836|            92|
|    2841|           115|
+--------+--------------+
only showing top 25 rows



In [ ]:
predictions.select("labelIndex", "prediction").show(10)

+----------+----------+
|labelIndex|prediction|
+----------+----------+
|       1.0|       1.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       1.0|
|       0.0|       1.0|
|       0.0|       0.0|
|       0.0|       1.0|
|       0.0|       0.0|
+----------+----------+
only showing top 10 rows



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.6866145701317019
Test Error = 0.3133854298682981


In [ ]:
df_val = assembler.transform(df_val)
df_test = assembler.transform(df_test)

In [ ]:
## VAL and TEST set
print(df_val.count())
print(df_test.count())
valpredictions = rfModel.transform(df_val)
testpredictions = rfModel.transform(df_test)

vallabels = valpredictions.select('prediction')
testlabels = testpredictions.select('prediction')
vallabels.show(40)

955
1086
+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       0.0|
|       1.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
+----------+
only showing top 40 rows



In [ ]:
values = {1:False, 0:True}
# vallabels = vallabels.replace(values, subset=['prediction'])
# testlabels = testlabels.replace(values, subset=['prediction'])
vallabels.show()

+----------+
|prediction|
+----------+
|       1.0|
|       0.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       1.0|
|       0.0|
|       1.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 20 rows



In [ ]:
from pyspark.sql.types import BooleanType

# Convert predictions to boolean
vallabels = vallabels.withColumn("prediction", vallabels.prediction.cast(BooleanType())).withColumn('prediction',initcap(col('prediction')))
testlabels = testlabels.withColumn("prediction", testlabels.prediction.cast(BooleanType())).withColumn('prediction',initcap(col('prediction')))
vallabels.show()

+----------+
|prediction|
+----------+
|      True|
|     False|
|      True|
|      True|
|      True|
|      True|
|      True|
|     False|
|      True|
|      True|
|     False|
|     False|
|     False|
|      True|
|     False|
|     False|
|     False|
|     False|
|     False|
|     False|
+----------+
only showing top 20 rows



In [ ]:
from google.colab import files
vallabels.write.mode('overwrite').csv('validation123.csv')
files.download('validation123.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
testlabels.write.mode('overwrite').csv('test123.csv')
files.download('test123.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## VAL and TEST set DONE

In [ ]:
from pyspark.sql.functions import col

# Convert label column to numeric type
training_data = train.withColumn("label", col("label").cast("double"))

# Split training data into train and test sets
train_data, test_data = training_data.randomSplit([0.7, 0.3], seed=42)

# Define the logistic regression model
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='numVotes', labelCol='label', maxIter=10)

# Define the parameter grid for tuning
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Define the evaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='label')

# Define the cross-validator
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           trainRatio=0.8,
                           seed=42)

# Fit the model to the training data
lr_model = tvs.fit(train_data)

# Make predictions on the test data
lr_predictions = lr_model.transform(test_data)

# Evaluate the model performance on the test data
test_auc = evaluator.evaluate(lr_predictions)
print('Test AUC:', test_auc)


In [ ]:
df_all = df_all.drop('tconst', 'primaryTitle', 'originalTitle', 'startYear', 'endYear')


In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Convert columns to double data type
df_all = df_all.select([df_all[col].cast('double').alias(col) for col in df_all.columns])
df_val = df_val.select([df_val[col].cast('double').alias(col) for col in df_val.columns])
df_test = df_test.select([df_test[col].cast('double').alias(col) for col in df_test.columns])

# Split the features and target variables for the training set
feature_cols = [col for col in df_all.columns if col != 'label']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
df_all = assembler.transform(df_all).select('features', 'label')
df_val = assembler.transform(df_val).select('features')
df_test = assembler.transform(df_test).select('features')

rf = RandomForestClassifier()
print("Number of rows in df_all:", df_all.count())
rf_model = rf.fit(df_all)

# Fit the model on the training data
rf_model = rf.fit(df_all)

# Predict on the validation set
y_val_pred = rf_model.transform(df_val)

# Evaluate the model using the MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
val_accuracy = evaluator.evaluate(y_val_pred)
print("Validation accuracy:", val_accuracy)


In [ ]:
from pyspark.sql.functions import count, isnan, when, col

# count the number of non-null rows in each column
df_all.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_all.columns]).show()

# check if the DataFrame is empty
if df_all.rdd.isEmpty():
    print("DataFrame is empty!")
else:
    print("DataFrame is not empty.")


In [ ]:
df_all.printSchema()


In [ ]:
columns_to_drop = ["primaryTitle", "originalTitle", "startYear", "endYear", "runtimeMinutes"]
df_all = df_all.drop(*columns_to_drop)
df_val = df_val.drop(*columns_to_drop)
df_test = df_test.drop(*columns_to_drop)


In [ ]:
from pyspark.sql.functions import col

# Cast the label column to a numeric type
df_all = df_all.withColumn("label", col("label").cast("double"))


In [ ]:
# Drop the existing 'features' column if it exists
# df_all = df_all.na.drop()


if 'features' in df_all.columns:
    df_all = df_all.drop('features')

# Combine the feature columns into a single vector column
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
df_all = assembler.transform(df_all)

# Train the logistic regression model on the training dataset
lr = LogisticRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(df_all)


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Train the Random Forest model on the training dataset
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
rf_model = rf.fit(train)

# Make predictions on the validation dataset
predictions = rf_model.transform(validation)

# Evaluate the model's accuracy on the validation dataset
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Validation Accuracy = {:.2f}%".format(accuracy*100))
